In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable

In [0]:
stream_df = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.schemaLocation", "/Volumes/gautham/gtk_scm/test_vlm/src/erp_px_cat_g1v2/")   # 🔹 stores inferred schema here
    .option("header", "true")
    .option("inferSchema", "true")                          # infer data types
    .option("cloudFiles.inferColumnTypes", "true")
    .option("cloudFiles.maxFilesPerTrigger", 100)
    .load("/Volumes/gautham/gtk_scm/test_vlm/src/erp_px_cat_g1v2/")
)

In [0]:
def process_batch(df, batch_id):
    df=df.withColumn("dwh_create_date",lit(current_timestamp()))
    target_table = DeltaTable.forName(spark, "gautham.gtk_scm.erp_px_cat_g1v2")

    # Define merge condition
    merge_condition = "tgt.ID = src.ID  "
    update_condition = """
            tgt.CAT <> src.CAT OR
            tgt.SUBCAT <> src.SUBCAT OR
            tgt.MAINTENANCE <> src.MAINTENANCE 
        """

    # 5Perform merge
    (
        target_table.alias("tgt")
        .merge(df.alias("src"), merge_condition)
        .whenMatchedUpdateAll(condition=update_condition)
        .whenNotMatchedInsertAll()
        .execute()
    )
        

In [0]:
stream_df.writeStream.foreachBatch(process_batch).option("checkpointLocation", "/Volumes/gautham/gtk_scm/test_vlm/chkp/erp_px_cat_g1v2/").trigger(availableNow=True).start().awaitTermination()